## Import Packages

This script makes use of the packages below.  
* "os" is only required if you want to specify a local folder in which to save an output (e.g., CSV or image file).
* 'sqlalchemy' is used to create a connection engine to our Postgres database.
* 'pandas' allows us to query our database using SQL while arraying our data calls into dataframes.
* 'numpy', 'seaborn', and 'matplotlib' are used for visualization purposes.

In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## Set working directory (optional).

In [2]:
# Set the working directory.
os.chdir('C:/Users/dnavarr/Documents/Python Scripts/used_cars/scripts')

## Connect to our database.
This code uses the 'create_engine' function from 'sqlalchemy'.

In [3]:
engine = create_engine('postgresql://[USER]@[IP]/[DATABASE]')

## Import our data.
We will make separate queries to our database to retreive data from our different tables.

For purposes of this exercise, I am making separate calls to each of our tables and later merging data using 'pandas'.

In a future iteration, we could make the joins at the import level using SQL.  

But for now, I'm doing this with Python/pandas.


### Import MarketCheck Honda data subset.

In [4]:
# I tested this script by querying only "Honda" vehicles. 

# If you want to query a different make, modify the "WHERE" clause.

marketcheck = pd.read_sql_query("""
SELECT
vin_ss,
vehicle_type_ss,
body_type_ss,
UPPER(make_ss) AS "make_ss",
UPPER(model_ss) AS "model_ss",
NULLIF(year_is, '')::int AS "year_is",
fuel_type_ss,
engine_size_ss,
transmission_ss,
doors_is,
cylinders_is,
interior_color_ss,
exterior_color_ss,
LPAD(zip_is, 5, '0') AS "zipcode",
NULLIF(price_fs, '')::int AS "price_fs",
NULLIF(miles_fs, '')::int AS "miles_fs",
latitude_fs,
longitude_fs,
city_ss,
state_ss,
dealer_id_is,
char_length(seller_comments_texts) AS "seller_comments_length",
char_length(options_texts) AS "options_length",
char_length(features_texts) AS "features_length",
dom_is
FROM listings
WHERE
  (upper(make_ss) LIKE 'HONDA' AND price_fs <>'')
  AND (upper(make_ss) LIKE 'HONDA' AND miles_fs <>'')
  AND (upper(make_ss) LIKE 'HONDA' AND model_ss <>'')
ORDER BY
  zipcode,
  state_ss
;
""", con=engine)

print(marketcheck.head())

              vin_ss vehicle_type_ss body_type_ss make_ss model_ss  year_is  \
0  3CZRM3H39FG709908             SUV          SUV   HONDA     CR-V     2015   
1  5FNYF6H52HB021773             SUV          SUV   HONDA    PILOT     2017   
2  2HGFB2F5XFH503740             Car        Sedan   HONDA    CIVIC     2015   
3  1HGCR2F82FA080126             Car        Sedan   HONDA   ACCORD     2015   
4  1HGCR2F82FA080126             Car        Sedan   HONDA   ACCORD     2015   

       fuel_type_ss engine_size_ss transmission_ss doors_is  ...   miles_fs  \
0  Regular Unleaded            2.4       Automatic        4  ...      35708   
1  Regular Unleaded            3.5                        4  ...      58628   
2  Regular Unleaded            1.8       Automatic        4  ...      41669   
3  Regular Unleaded            2.4       Automatic        4  ...      18539   
4  Regular Unleaded            2.4       Automatic        4  ...      18539   

  latitude_fs longitude_fs    city_ss  state_ss  d

### Import City Data

MarketCheck's vehicle listing data includes ZIP code, city, and state information.  

The 'cities_extended' dataset will allow us to identify corresponding county information.

In [5]:
cities = pd.read_sql_query("""
                               SELECT *
                               FROM cities_extended;
                               """, con=engine)

print(cities.head())

        city state_code    zip   latitude   longitude     county
0   Adjuntas         PR  00601    18.1788    -66.7516   Adjuntas
1     Aguada         PR  00602  18.381389  -67.188611     Aguada
2  Aguadilla         PR  00603    18.4554    -67.1308  Aguadilla
3  Aguadilla         PR  00604    18.4812    -67.1467  Aguadilla
4  Aguadilla         PR  00605  18.429444  -67.154444  Aguadilla


### Import Census FIPS Code Data

Because county names might be misspelled across different datasets (e.g., "Saint" vs. "St." vs "St"), we will use the national FIPS standard.  

Each county in each state is coded with its own unique FIPS code.

In [6]:
census_fips = pd.read_sql_query("""
SELECT
state,   
county_name,  
LPAD(fips, 5, '0') AS "fips_code"
FROM census_fips;""",
con=engine)

print(census_fips.head())

  state county_name fips_code
0    AL     Autauga     01001
1    AL     Baldwin     01003
2    AL     Barbour     01005
3    AL        Bibb     01007
4    AL      Blount     01009


### Import BEA Regions

As we would like to consider geographic regions for analysis, we will use the BEA-defined regions.

In [7]:
regions = pd.read_sql_query("""
                                SELECT *
                                FROM bea_regions;
                                """, con=engine)

print(regions.head())

   state_code     state_name state_abbrev  region_code         region_name  \
0           9    Connecticut           CT            1  New England Region   
1          23          Maine           ME            1  New England Region   
2          25  Massachusetts           MA            1  New England Region   
3          33  New Hampshire           NH            1  New England Region   
4          44   Rhode Island           RI            1  New England Region   

  region_abbrev  region_id  
0          NENG         91  
1          NENG         91  
2          NENG         91  
3          NENG         91  
4          NENG         91  


### Import Census/BLS metropolitan and non-metropolitan data.

We will bring in a dataset that will allow us to identify each listing's corresponding MSA designation.

In [8]:
locality = pd.read_sql_query("""
                               SELECT *
                               FROM locality;
                               """, con=engine)

print(locality.head())

  zip_code   fips        locality_name        state     cbsa  \
0    00601  72001   Adjuntas Municipio  Puerto Rico  7200001   
1    00601  72141     Utuado Municipio  Puerto Rico  7200001   
2    00602  72003     Aguada Municipio  Puerto Rico    10380   
3    00603  72005  Aguadilla Municipio  Puerto Rico    10380   
4    00606  72093    Maricao Municipio  Puerto Rico  7200001   

                              msa_name  
0   Puerto Rico nonmetropolitan area 1  
1   Puerto Rico nonmetropolitan area 1  
2  Aguadilla-Isabela-San Sebastian, PR  
3  Aguadilla-Isabela-San Sebastian, PR  
4   Puerto Rico nonmetropolitan area 1  


#### We will do a bit of wrangling to reduce the scope of our Census MSA data.

In [9]:
# Number of rows:
len(locality)

634019

In [10]:
# Drop place names from the locality dataframe, 
# otherwise our data explodes (as county can have multiple zips, cities, etc.)
locality = locality.drop('locality_name', axis=1)

# Define whether an MSA is a Micropolitan Division.
locality['msa_type'] = np.where(locality['msa_name'].str.contains("Division"), "Division", "")

# Drop if it is a Division as this will create a lot of duplicative data.
# For example, there are 2 Micropolitan Divisions within the DC/MD/VA Metropolitan MSA.
# By keeping the Micropolitan Divisions, we'd be replicating each DC-area car listing by 3.
locality = locality[locality.msa_type != "Division"]

# Designate whether the MSA is a Metropolitan or Non-Metropolitan
locality['msa_type'] = np.where(locality['msa_name'].str.contains("nonmetropolitan"), 
                                "Non-Metropolitan", "Metropolitan")

# We will drop duplicates.
locality = locality.drop_duplicates()

# Number of rows:
len(locality)

42451

In [11]:
print(locality.head())

  zip_code   fips        state     cbsa                             msa_name  \
0    00601  72001  Puerto Rico  7200001   Puerto Rico nonmetropolitan area 1   
1    00601  72141  Puerto Rico  7200001   Puerto Rico nonmetropolitan area 1   
2    00602  72003  Puerto Rico    10380  Aguadilla-Isabela-San Sebastian, PR   
3    00603  72005  Puerto Rico    10380  Aguadilla-Isabela-San Sebastian, PR   
4    00606  72093  Puerto Rico  7200001   Puerto Rico nonmetropolitan area 1   

           msa_type  
0  Non-Metropolitan  
1  Non-Metropolitan  
2      Metropolitan  
3      Metropolitan  
4  Non-Metropolitan  


## Bring in SOI data

In [12]:
####  SOI Data
soi_data = pd.read_sql_query("""
                                 SELECT
                                 STATE,
                                 ZIPCODE,
                                 AGI_STUB,
                                 N1,
                                 MARS1,
                                 MARS2,
                                 N2,
                                 NUMDEP,
                                 RAL,
                                 RAC,
                                 ELDERLY,
                                 A00100,
                                 A02650,
                                 N26270,
                                 A03220,
                                 N03300,
                                 N03210,
                                 N03230,
                                 SCHF,
                                 N18800,
                                 N19300,
                                 N19700                       
                                 FROM irs_soi_full;
                                 """, con=engine)

print(soi_data.head())

  state zipcode  agi_stub      n1   mars1   mars2       n2  numdep    ral  \
0    AL       0         1  815440  477700  105350  1296920  491310  31550   
1    AL       0         2  495830  211930  142340   996240  360480  11770   
2    AL       0         3  263390   83420  137870   584000  182880    150   
3    AL       0         4  167190   29420  124060   421720  130160      0   
4    AL       0         5  217440   20240  188080   601040  195990      0   

      rac   ...      a02650  n26270  a03220  n03300  n03210  n03230  schf  \
0  225140   ...    10787121    7900     308      40   17110    6620  8310   
1  118460   ...    18020908   10340    3476      90   41490    2340  8920   
2   41910   ...    16351320   11580    2146     190   29110    2150  8470   
3   18560   ...    14646693    8780    2323     110   20610     360  5510   
4   11620   ...    29696755   24980    3192    1310   22870    3450  8180   

   n18800  n19300  n19700  
0   25020   20950   36370  
1   64360   60550 

## Dataframe Merging (joins)

In [13]:
# Merge our listings dataframe with our cities data using ZIP code and state.
listings = pd.merge(marketcheck, cities, left_on=['state_ss', 'zipcode'], right_on=['state_code', 'zip'])

# Merge listings with Census FIPS codes.
listings = pd.merge(listings, census_fips, left_on=['state_ss', 'county',], right_on=['state', 'county_name'])

# Merge regions into our listings dataframe.
listings = pd.merge(listings, regions, left_on='state_code', right_on='state_abbrev')


In [14]:
# Merge our listings dataframe with the locality dataset.
# Even though we eliminated Micropolitan divisions, our data still explodes due to the high number of zip codes.
# This step might be memory intensive as it creates a dataframe with over 11 million rows.
listings = pd.merge(listings, locality, left_on='fips_code', right_on='fips')

# We drop duplicate columns that resulted from our dataframe merges.
listings = listings.drop(['zip', 'zip_code', 'city', 'state_code_x', 'state_x', 
                          'county_name', 'state_code_y', 'state_name', 'state_abbrev', 
                          'fips', 'state_y'], axis=1)

# We drop duplicate rows.  This should bring our dataframe down from 11 million rows to about 300,000 (for Honda).
listings = listings.drop_duplicates()

In [15]:
# Merge county names to SOI data.
soi_data = pd.merge(soi_data, cities, left_on=['state', 'zipcode'], right_on=['state_code', 'zip'])

soi_data = soi_data.groupby(['zipcode']).sum().reset_index()

listings = pd.merge(listings, soi_data, left_on=['zipcode'], right_on=['zipcode'])

In [16]:
#### CLEAN-UP for Memory; 
del(census_fips, cities, marketcheck, regions, locality)

In [17]:
print(listings.head())

print(len(listings))

              vin_ss vehicle_type_ss body_type_ss make_ss model_ss  year_is  \
0  19XFB2F84FE201705             Car        Sedan   HONDA    CIVIC     2015   
1  5J6RM4H95GL033651             SUV          SUV   HONDA     CR-V     2016   
2  2HKRM4H54DH671649             SUV          SUV   HONDA     CR-V     2013   
3  2HGFC4B01HH310897             Car        Coupe   HONDA    CIVIC     2017   
4  SHHFK7H59JU404168             Car    Hatchback   HONDA    CIVIC     2018   

       fuel_type_ss engine_size_ss                     transmission_ss  \
0  Regular Unleaded            1.8                           Automatic   
1  Regular Unleaded            2.4                           Automatic   
2  Regular Unleaded            2.4                           Automatic   
3  Regular Unleaded            2.0                           Automatic   
4          Gasoline            1.5  Continuously Variable Transmission   

  doors_is  ...    a02650 n26270 a03220 n03300  n03210  n03230 schf n18800  \
0 

## Drop duplicates.

In [18]:
#sorts listings from most days on the market to least
listings_sorted = listings.sort_values('dom_is', ascending = False)

#drops duplicate vin numbers keeping first (longest on market) vin
listings = listings_sorted.drop_duplicates(subset='vin_ss', keep="first")

In [19]:
print(len(listings))

75056


## Label Encoding

We will encode our categorical data into numerics so that we can use scikit-learn and seaborn packages.

In [20]:
### Label Encoding
from sklearn import preprocessing

# We create a label (category) encoder object using scikit-learn's "le" 
# naming convention (based on their documentation).
le = preprocessing.LabelEncoder()

# Fit the encoder to the pandas column
le.fit(listings['model_ss'])

# View the labels
list(le.classes_)

# Transform the Categories into Integers
le.transform(listings['model_ss'])

# Append to our dataframe.
listings['model_ss_encoded'] = le.transform(listings['model_ss'])

C:\Users\dnavarr\AppData\Local\Continuum\anaconda3\envs\alpastor\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Repeat label encoding process for each categorical variable.

In [21]:
# make_ss
le = preprocessing.LabelEncoder()
le.fit(listings['make_ss'])
list(le.classes_)
le.transform(listings['make_ss'])
listings['make_ss_encoded'] = le.transform(listings['make_ss'])

# body_type_ss
le = preprocessing.LabelEncoder()
le.fit(listings['body_type_ss'])
list(le.classes_)
le.transform(listings['body_type_ss'])
listings['body_type_ss_encoded'] = le.transform(listings['body_type_ss'])

# vehicle_type_ss
le = preprocessing.LabelEncoder()
le.fit(listings['vehicle_type_ss'])
list(le.classes_)
le.transform(listings['vehicle_type_ss'])
listings['vehicle_type_ss_encoded'] = le.transform(listings['vehicle_type_ss'])

# fuel_type_ss
le = preprocessing.LabelEncoder()
le.fit(listings['fuel_type_ss'])
list(le.classes_)
le.transform(listings['fuel_type_ss'])
listings['fuel_type_ss_encoded'] = le.transform(listings['fuel_type_ss'])

# engine_size_ss
le = preprocessing.LabelEncoder()
le.fit(listings['engine_size_ss'])
list(le.classes_)
le.transform(listings['engine_size_ss'])
listings['engine_size_ss_encoded'] = le.transform(listings['engine_size_ss'])

# transmission_ss
le = preprocessing.LabelEncoder()
le.fit(listings['transmission_ss'])
list(le.classes_)
le.transform(listings['transmission_ss'])
listings['transmission_ss_encoded'] = le.transform(listings['transmission_ss'])

# doors_is
le = preprocessing.LabelEncoder()
le.fit(listings['doors_is'])
list(le.classes_)
le.transform(listings['doors_is'])
listings['doors_is_encoded'] = le.transform(listings['doors_is'])

# cylinders_is
le = preprocessing.LabelEncoder()
le.fit(listings['cylinders_is'])
list(le.classes_)
le.transform(listings['cylinders_is'])
listings['cylinders_is_encoded'] = le.transform(listings['cylinders_is'])

# interior_color_ss
le = preprocessing.LabelEncoder()
le.fit(listings['interior_color_ss'])
list(le.classes_)
le.transform(listings['interior_color_ss'])
listings['interior_color_ss_encoded'] = le.transform(listings['interior_color_ss'])

# exterior_color_ss
le = preprocessing.LabelEncoder()
le.fit(listings['exterior_color_ss'])
list(le.classes_)
le.transform(listings['exterior_color_ss'])
listings['exterior_color_ss_encoded'] = le.transform(listings['exterior_color_ss'])

# state_ss
le = preprocessing.LabelEncoder()
le.fit(listings['state_ss'])
list(le.classes_)
le.transform(listings['state_ss'])
listings['state_ss_encoded'] = le.transform(listings['state_ss'])

# county_name
le = preprocessing.LabelEncoder()
le.fit(listings['county'])
list(le.classes_)
le.transform(listings['county'])
listings['county_encoded'] = le.transform(listings['county'])

# fips_code
le = preprocessing.LabelEncoder()
le.fit(listings['fips_code'])
list(le.classes_)
le.transform(listings['fips_code'])
listings['fips_code_encoded'] = le.transform(listings['fips_code'])

# msa_type
le = preprocessing.LabelEncoder()
le.fit(listings['msa_type'])
list(le.classes_)
le.transform(listings['msa_type'])
listings['msa_type_encoded'] = le.transform(listings['msa_type'])

# msa_name
le = preprocessing.LabelEncoder()
le.fit(listings['msa_name'])
list(le.classes_)
le.transform(listings['msa_name'])
listings['msa_name_encoded'] = le.transform(listings['msa_name'])

C:\Users\dnavarr\AppData\Local\Continuum\anaconda3\envs\alpastor\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\dnavarr\AppData\Local\Continuum\anaconda3\envs\alpastor\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\dnavarr\AppData\Local\Continuum\anaconda3\envs\alpastor\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

## Z-Scores

In [22]:
pd.options.mode.chained_assignment = None

listings['zscore'] = np.abs(stats.zscore(listings['price_fs']))
listings.describe()

,year_is,price_fs,miles_fs,seller_comments_length,options_length,features_length,dom_is,region_code,region_id,agi_stub,...,doors_is_encoded,cylinders_is_encoded,interior_color_ss_encoded,exterior_color_ss_encoded,state_ss_encoded,county_encoded,fips_code_encoded,msa_type_encoded,msa_name_encoded,zscore
count,75056.000000,75056.000000,7.505600e+04,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.0,...,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000,75056.000000
mean,2013.311434,17352.138510,6.510163e+04,660.293980,746.204674,2380.702396,97.369204,4.942483,94.942483,21.0,...,1.896797,5.133074,293.879756,832.579407,18.549963,530.060355,738.529738,0.100898,217.623321,0.362642
std,3.918001,17504.900252,5.300020e+04,763.294264,1565.002978,2024.648567,152.336929,1.801112,1.801112,0.0,...,0.375355,1.018952,268.772507,517.372970,11.979641,285.538797,493.988324,0.301196,136.067764,0.931935
min,2000.000000,301.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,2.000000,92.000000,21.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000008
25%,2011.000000,11500.000000,2.680100e+04,0.000000,0.000000,182.000000,24.000000,3.000000,93.000000,21.0,...,2.000000,5.000000,0.000000,330.000000,6.000000,316.000000,198.000000,0.000000,80.000000,0.133484
50%,2015.000000,16995.000000,4.593950e+04,424.000000,0.000000,2548.000000,54.000000,5.000000,95.000000,21.0,...,2.000000,5.000000,219.000000,777.000000,20.000000,532.000000,818.000000,0.000000,224.000000,0.286342
75%,2016.000000,21842.500000,9.541525e+04,1109.000000,1047.000000,3574.000000,112.000000,6.000000,96.000000,21.0,...,2.000000,6.000000,527.000000,1335.000000,30.000000,787.000000,1167.000000,0.000000,332.000000,0.534548
max,2018.000000,1000000.000000,1.442200e+06,10042.000000,14982.000000,17853.000000,2057.000000,8.000000,98.000000,21.0,...,3.000000,6.000000,998.000000,1691.000000,40.000000,1040.000000,1622.000000,1.000000,469.000000,56.135962


## Identify Outliers

In [23]:
#  Create Outlier column
listings['outlier'] = listings['zscore'] > 3

listings['outlier']

231121    False
66537     False
144754    False
232379    False
172683    False
12751     False
14168     False
27592     False
27590     False
27535     False
27519     False
27584     False
27630     False
27643     False
27660     False
27662     False
27663     False
27664     False
27673     False
27568     False
27594     False
266471    False
214422    False
27542     False
251740    False
258879    False
160748    False
210988    False
272387    False
144758    False
          ...  
183950    False
227160    False
34543     False
226674    False
184186    False
227064    False
227781    False
272677    False
183244    False
149073    False
183176    False
183275    False
182910    False
182909    False
182794    False
228348    False
183617    False
183737    False
183642    False
111073    False
184687    False
114491    False
184620    False
20267     False
115145    False
272843    False
20364     False
184515    False
184554    False
184606    False
Name: outlier, Length: 7

In [24]:
# Drop outliers

listings = listings.drop(listings[listings.outlier == True].index)

listings.describe()

,year_is,price_fs,miles_fs,seller_comments_length,options_length,features_length,dom_is,region_code,region_id,agi_stub,...,doors_is_encoded,cylinders_is_encoded,interior_color_ss_encoded,exterior_color_ss_encoded,state_ss_encoded,county_encoded,fips_code_encoded,msa_type_encoded,msa_name_encoded,zscore
count,75025.000000,75025.000000,7.502500e+04,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.0,...,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000,75025.000000
mean,2013.312776,17086.118827,6.508791e+04,660.350896,746.490530,2380.489530,97.351390,4.943086,94.943086,21.0,...,1.896808,5.133076,293.976794,832.639214,18.550297,530.084505,738.575701,0.100860,217.666005,0.347595
std,3.917482,7692.683039,5.299954e+04,763.400546,1565.251365,2024.834597,152.314161,1.800797,1.800797,0.0,...,0.375335,1.018944,268.769999,517.371152,11.980266,285.545988,494.036142,0.301145,136.050532,0.269320
min,2000.000000,301.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,2.000000,92.000000,21.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000008
25%,2011.000000,11500.000000,2.679600e+04,0.000000,0.000000,182.000000,24.000000,3.000000,93.000000,21.0,...,2.000000,5.000000,0.000000,330.000000,6.000000,315.000000,198.000000,0.000000,80.000000,0.133270
50%,2015.000000,16995.000000,4.592500e+04,423.000000,0.000000,2548.000000,54.000000,5.000000,95.000000,21.0,...,2.000000,5.000000,219.000000,777.000000,20.000000,532.000000,818.000000,0.000000,224.000000,0.285857
75%,2016.000000,21817.000000,9.536200e+04,1110.000000,1048.000000,3574.000000,112.000000,6.000000,96.000000,21.0,...,2.000000,6.000000,527.000000,1335.000000,30.000000,787.000000,1167.000000,0.000000,332.000000,0.534548
max,2018.000000,69706.000000,1.442200e+06,10042.000000,14982.000000,17853.000000,2057.000000,8.000000,98.000000,21.0,...,3.000000,6.000000,998.000000,1691.000000,40.000000,1040.000000,1622.000000,1.000000,469.000000,2.990832


# Save dataframe output.

In [25]:
import pickle

listings.to_pickle("Second_Analysis_Pickle.pkl")